In [ ]:
import numpy as np
import decimal
import matplotlib.pyplot as plt
import pandas as pd
from scipy.spatial.distance import cityblock
import glob
import statistics as st

from apollon.som.som import IncrementalMap
from apollon.som.utilities import best_match
from pathlib import Path
import pickle # for exporting and importing and exporting scaler and som

import plotly.graph_objects as go
from sklearn import preprocessing #for normierung

## Prepare features

In [ ]:
# dictionary of all features:
features = {1: 'PhaseSpace',
            2: 'ChannelCorrelation',
            3: 'PeakMeter',
            4: 'RMS',
            5: 'CrestFactor',
            6: 'PhaseSpaceLow',
            7: 'ChannelCorrelationLow',
            8: 'PeakMeterLow',
            9: 'RMSLow',
            10: 'CrestFactorLow',
            11: 'PhaseSpaceMid',
            12: 'ChannelCorrelationMid',
            13: 'PeakMeterMid',
            14: 'RMSMid',
            15: 'CrestFactorMid',
            16: 'PhaseSpaceHigh',
            17: 'ChannelCorrelationHigh',
            18: 'PeakMeterHigh',
            19: 'RMSHigh',
            20: 'CrestFactorHigh',
            21: 'bpm'}

# selection of preferred features
nfeatures = {0: 21,
             1: 1,
             2: 2,
             3: 5#,4:3, 5:4, 6:6, 7:7, 8:8, 9:9, 10:10, 11:11, 12:12, 13:13, 14:14, 15:15, 16:16, 17:17, 18:18, 19:19, 20:20
            }
# shall we take each feratures' 'mean', 'median', standard deviation ('std'), skewness ('skew'), or kurtosis ('kurt')?
what=['median','median','median','median']
numfeatures = len(nfeatures)

measCsv=r'C:\Users\t_ziemer\Desktop\techno-csv'#top folder that contains one subfolder for each category that we want to mark with its own color later
folds=glob.glob(str(measCsv+'\\*'))

## Data Preparation

## Normalization

## Fit SOM

In [ ]:
# Load SOM and Scaler:
sompath = r'C:\python\models\2025-02-24-med-bpm-phsp-chacorr-crest.sav'
scalerpath = r'C:\python\models\2025-02-24-med-bpm-phsp-chacorr-crest.pkl'
som = pickle.load(open(sompath, 'rb'))
scaler = pickle.load(open(scalerpath,'rb'))

## Results

In [ ]:
# load u-matrix and choose color scheme for item point cloud:
um = som.umatrix()

# colors for various styles (subgenres):
#colrs=['lightcoral','blue','red','indianred','magenta','darkorange','slateblue','hotpink','blueviolet','lightpink','dodgerblue','deepskyblue','deeppink','royalblue','tomato','skyblue','mediumblue','royalblue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red']

# colors for nations and various years:
#colrs=['red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red','blue','red']

# colors for nations only:
colrs=['blue','red']

In [ ]:
# Assign each item to its Best Matching Unit (BMU) on the SOM:
import random
msz=5
trace1=go.Heatmap(z=um, colorscale='viridis', showscale = False, hoverinfo= 'none')
data=[trace1]

n=0
for fold in folds:
    files = glob.glob(str(fold+'\\*.csv'))
    numfiles = len(files)
    lal1 = np.empty((numfiles, numfeatures))
    nameS = []
    nameT = []
    for j, f in enumerate(files):
        la = pd.read_csv(f, index_col=0)
        nameS.append(Path(f).stem)
        nameT.append(Path(f).stem[0])
        for dt, i in enumerate(nfeatures):
            if what[dt]=='median':
                lal1[j][i-1] = la[features[nfeatures[i]]].median()
            elif what[dt]=='mean':
                lal1[j][i-1] = la[features[nfeatures[i]]].mean()
            elif what[dt]=='std':
                lal1[j][i-1] = la[features[nfeatures[i]]].std()
            elif what[dt]=='skew':
                lal1[j][i-1] = la[features[nfeatures[i]]].skew()
            elif what[dt]=='kurt':
                lal1[j][i-1] = la[features[nfeatures[i]]].kurtosis()
    bmu, err = best_match(som.weights, scaler.transform(lal1), som.metric)
    py, px = np.unravel_index(bmu, som.shape)
    trace=go.Scatter(x=px+random.uniform(-0.4,0.4),y=py+random.uniform(-0.4,0.4), mode='markers', name=Path(fold).stem, hoverinfo='text', text = nameS, marker_size=msz, marker_color=colrs[n])
    data.append(trace)
    n=n+1

In [ ]:
# plot the U-Matrix of the SOM and all items:
lay=go.Layout(width=900, height=800)
fig=go.Figure(data=data, layout=lay)
# save as vector graphic for papers:
#fig.savefig('SOM-BPM-PhSp-ChCorr-Crest-Genres.pdf', format='pdf',bbox_inches='tight')
fig.update_xaxes(range=[-1, 115])  # Set x-axis range
fig.update_yaxes(range=[-1, 115])  # Set y-axis range
fig.show()

In [ ]:
# Export U-Matrix of the SOM as an interactive HTML file:
fig.write_html(r'C:\python\maps\2025-03-10-med-bpm-phsp-chacorr-crest-Nations+years-scale.html')

## Component planes

In [ ]:
#Plot component planes. Each subplot is a feature selected above
fig, (axs) = plt.subplots(2, 2, figsize=(8, 8), sharex=True, sharey=True)
fig.tight_layout()
for i, (fn, ax) in enumerate(zip(nfeatures, axs.flatten())):
    ax.set_title(features[nfeatures[fn]])
    ax.imshow(som.weights[:, i].reshape(som.shape), origin='lower', aspect='equal')

In [ ]:
# export component planes as a PNG File:
fig.savefig(r'C:\python\maps\2025-02-24-med-bpm-phsp-chacorr-crest.png',bbox_inches='tight')